In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py

import tensorflow as tf
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices(device_type="GPU")[0], True)
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from tqdm import tqdm

from msfm.utils import tfrecords, files, parameters, cross_statistics
from msfm.fiducial_pipeline import FiducialPipeline
from msfm.grid_pipeline import GridPipeline
from msfm.utils.input_output import read_yaml

In [3]:
with_lensing = True
with_clustering = True

# fiducial

In [4]:
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_0000.tfrecord"
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_0001.tfrecord"
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_00??.tfrecord"
tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_00??.tfrecord"

# conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias_debug.yaml"
conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias.yaml"
conf = files.load_config(conf)

# params = ["Om", "s8", "Ob", "H0", "ns", "w0", "Aia", "n_Aia", "bg", "n_bg"]
# params = ["Om", "s8", "w0", "Aia", "n_Aia", "bg", "n_bg"]
params = ["Om", "s8"]
# params = ["Om", "Aia"]
# params = ["s8", "Aia"]
# params = ["Om", "bg"]
# params = ["Om"]
# params = ["Aia"]
# params = ["bg"]

In [7]:
fidu_pipe = FiducialPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
    return_cls=False,
)
fidu_dset = fidu_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    local_batch_size=4,
    n_readers=8,
    n_prefetch=0,
    is_eval=False,
)

for x in tqdm(fidu_dset.take(n_batches), total=n_batches, desc=f"return_cls = {return_cls}, return_maps = {return_maps}"):
    pass

24-08-28 05:53:16     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:53:17     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:53:17 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:53:17 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:53:17 fiducial_pip INF   Including noise_indices = [0] 
24-08-28 05:53:17 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-28 05:53:17 fiducial_pip INF   Interleaving with n_readers = 8 


ic| *args: False


24-08-28 05:53:17 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-28 05:53:17 fiducial_pip INF   Batching into 4 elements locally with drop_remainder = True 
24-08-28 05:53:17 fiducial_pip WAR   Tracing _augmentations 
24-08-28 05:53:17 fiducial_pip INF   Running on the data_vectors.keys() = dict_keys(['kg_fiducial', 'dg_fiducial', 'kg_delta_Om_m', 'dg_delta_Om_m', 'kg_delta_Om_p', 'dg_delta_Om_p', 'kg_delta_s8_m', 'dg_delta_s8_m', 'kg_delta_s8_p', 'dg_delta_s8_p', 'i_example', 'sn', 'pn', 'i_noise']) 
24-08-28 05:53:17 fiducial_pip WAR   Tracing _lensing_augmentations 
24-08-28 05:53:17 fiducial_pip WAR   Tracing _clustering_augmentations 
24-08-28 05:53:17 fiducial_pip INF   Successfully generated the fiducial training set with element_spec (TensorSpec(shape=(20, 460800, 8), dtype=tf.float32, name=None), NoneTensorSpec(), (TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))) 


return_cls = False, return_maps = True: 100%|██████████| 100/100 [02:01<00:00,  1.21s/it]


In [5]:
n_batches = 100

for return_cls in [True, False]:
    for return_maps in [True, False]:
        if return_cls or return_maps:
            print(f"return_cls = {return_cls}, return_maps = {return_maps}")
            
            fidu_pipe = FiducialPipeline(
                conf=conf,
                params=params,
                with_lensing=with_lensing,
                with_clustering=with_clustering,
                with_padding=True,
                apply_norm=False,
                return_maps=return_maps,
                return_cls=return_cls,
            )
            fidu_dset = fidu_pipe.get_dset(
                tfr_pattern=tfr_pattern,
                local_batch_size=4,
                n_readers=8,
                n_prefetch=0,
                is_eval=False,
            )
            
            for x in tqdm(fidu_dset.take(n_batches), total=n_batches, desc=f"return_cls = {return_cls}, return_maps = {return_maps}"):
                pass

return_cls = True, return_maps = True
24-08-28 05:48:03     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:48:03     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:48:10 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:48:10 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:48:10 fiducial_pip INF   Including noise_indices = [0] 
24-08-28 05:48:10 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-28 05:48:10 fiducial_pip INF   Interleaving with n_readers = 8 


ic| *args: True


24-08-28 05:48:14 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-28 05:48:14 fiducial_pip INF   Batching into 4 elements locally with drop_remainder = True 
24-08-28 05:48:15 fiducial_pip WAR   Tracing _augmentations 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
24-08-28 05:48:17

return_cls = True, return_maps = True: 100%|██████████| 100/100 [01:57<00:00,  1.18s/it]


return_cls = True, return_maps = False
24-08-28 05:50:15     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:50:15     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:50:16 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:50:16 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:50:16 fiducial_pip INF   Including noise_indices = [0] 
24-08-28 05:50:17 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-28 05:50:17 fiducial_pip INF   Interleaving with n_readers = 8 


...

return_cls = True, return_maps = False: 100%|██████████| 100/100 [01:29<00:00,  1.12it/s]


return_cls = False, return_maps = True
24-08-28 05:51:47     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:51:47     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:51:48 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:51:48 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:51:48 fiducial_pip INF   Including noise_indices = [0] 
24-08-28 05:51:48 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-28 05:51:48 fiducial_pip INF   Interleaving with n_readers = 8 


ic| *args: False


24-08-28 05:51:48 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-28 05:51:48 fiducial_pip INF   Batching into 4 elements locally with drop_remainder = True 
24-08-28 05:51:48 fiducial_pip WAR   Tracing _augmentations 
24-08-28 05:51:48 fiducial_pip INF   Running on the data_vectors.keys() = dict_keys(['kg_fiducial', 'dg_fiducial', 'kg_delta_Om_m', 'dg_delta_Om_m', 'kg_delta_Om_p', 'dg_delta_Om_p', 'kg_delta_s8_m', 'dg_delta_s8_m', 'kg_delta_s8_p', 'dg_delta_s8_p', 'i_example', 'sn', 'pn', 'i_noise']) 
24-08-28 05:51:48 fiducial_pip WAR   Tracing _lensing_augmentations 
24-08-28 05:51:49 fiducial_pip WAR   Tracing _clustering_augmentations 


KeyError: in user code:

    File "/global/u2/a/athomsen/multiprobe-simulation-forward-model/msfm/fiducial_pipeline.py", line 406, in _augmentations  *
        cl.append(data_vectors[f"cl_{label}"])

    KeyError: 'cl_fiducial'


In [ ]:
fidu_pipe = FiducialPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
)

data_vec_pix = fidu_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = fidu_pipe.n_z_metacal + fidu_pipe.n_z_maglim
n_z_metacal = fidu_pipe.n_z_metacal
n_noise = conf["analysis"]["fiducial"]["n_noise_per_example"]
fidu_batch_size = 16

fidu_dset = fidu_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=fidu_batch_size,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

In [ ]:
%%time
n_batches = 100
for x in tqdm(fidu_dset.take(n_batches), total=n_batches):
    pass

In [9]:
fidu_pipe = FiducialPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=False,
)

data_vec_pix = fidu_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = fidu_pipe.n_z_metacal + fidu_pipe.n_z_maglim
n_z_metacal = fidu_pipe.n_z_metacal
n_noise = conf["analysis"]["fiducial"]["n_noise_per_example"]
fidu_batch_size = 16

fidu_dset = fidu_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=fidu_batch_size,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

24-08-16 00:16:17     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:16:18     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:16:18 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-16 00:16:18 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-16 00:16:18 fiducial_pip INF   Including noise_indices = [0, 1] 
24-08-16 00:16:18 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-16 00:16:18 fiducial_pip INF   Interleaving with n_readers = 1 
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:5", shape=(), dtype=int64)
24-08-16 00:16:18 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-16 00:16:18 fiducial_pip INF   Batching into 16 elements locally with drop_

In [10]:
%%time
n_batches = 100
for x in tqdm(fidu_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [00:44<00:00,  2.23it/s]

CPU times: user 1min 5s, sys: 1min 18s, total: 2min 23s
Wall time: 44.8 s


# grid

In [4]:
tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/linear_bias/tfrecords/grid/DESy3_grid_dmb_00??.tfrecord"

conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias.yaml"
conf = files.load_config(conf)

params = ["Om", "s8", "Ob", "H0", "ns", "w0", "Aia", "n_Aia", "bg", "n_bg"]

In [17]:
grid_pipe.return_cls

False

In [22]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
    return_cls=False,
)
grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    local_batch_size=32,
    n_readers=8,
    n_prefetch=0,
    is_eval=False,
)

# for x in tqdm(grid_dset.take(n_batches), total=n_batches):
#     pass

24-08-28 05:24:18     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:24:18     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:24:19 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:24:19 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:24:19 grid_pipelin INF   Including noise_indices = [0] 
24-08-28 05:24:19 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-28 05:24:19 grid_pipelin INF   Interleaving with n_readers = 8 


ic| *args: False


dict_keys(['cosmo', 'i_sobol', 'i_example', 'kg', 'dg'])
24-08-28 05:24:19 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-28 05:24:19 grid_pipelin INF   Batching into 32 elements locally 
24-08-28 05:24:19 grid_pipelin WAR   Tracing _augmentations 
24-08-28 05:24:19 grid_pipelin INF   Running on the data_vectors.keys() = dict_keys(['cosmo', 'i_sobol', 'i_example', 'kg', 'dg', 'i_noise']) 
24-08-28 05:24:19 grid_pipelin INF   Successfully generated the grid validation set with element_spec (TensorSpec(shape=(32, 460800, 8), dtype=tf.float32, name=None), NoneTensorSpec(), TensorSpec(shape=(32, 10), dtype=tf.float32, name=None), (TensorSpec(shape=(32,), dtype=tf.int64, name=None), TensorSpec(shape=(32,), dtype=tf.int64, name=None), TensorSpec(shape=(32,), dtype=tf.int32, name=None))) 


In [23]:
n_batches = 100

for return_cls in [True, False]:
    for return_maps in [True, False]:
        if return_cls or return_maps:
            print(f"return_cls = {return_cls}, return_maps = {return_maps}")
            
            grid_pipe = GridPipeline(
                conf=conf,
                params=params,
                with_lensing=with_lensing,
                with_clustering=with_clustering,
                with_padding=True,
                apply_norm=False,
                return_maps=return_maps,
                return_cls=return_cls,
            )
            grid_dset = grid_pipe.get_dset(
                tfr_pattern=tfr_pattern,
                local_batch_size=32,
                n_readers=8,
                n_prefetch=0,
                is_eval=False,
            )
            
            for x in tqdm(grid_dset.take(n_batches), total=n_batches, desc=f"return_cls = {return_cls}, return_maps = {return_maps}"):
                pass

return_cls = True, return_maps = True
24-08-28 05:24:32     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:24:33     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:24:33 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:24:33 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:24:33 grid_pipelin INF   Including noise_indices = [0] 
24-08-28 05:24:33 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-28 05:24:33 grid_pipelin INF   Interleaving with n_readers = 8 
dict_keys(['cosmo', 'i_sobol', 'i_example', 'kg', 'dg', 'cl'])
24-08-28 05:24:33 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-28 05:24:33 grid_pipelin INF   Batching into 32 elements 

return_cls = True, return_maps = True: 100%|██████████| 100/100 [01:43<00:00,  1.04s/it]

return_cls = True, return_maps = False
24-08-28 05:26:17     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 


24-08-28 05:26:18     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:26:18 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:26:18 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:26:18 grid_pipelin INF   Including noise_indices = [0] 
24-08-28 05:26:18 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-28 05:26:18 grid_pipelin INF   Interleaving with n_readers = 8 
dict_keys(['cosmo', 'i_sobol', 'i_example', 'cl'])
24-08-28 05:26:18 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-28 05:26:18 grid_pipelin INF   Batching into 32 elements locally 
24-08-28 05:26:18 grid_pipelin WAR   Tracing _augmentations 
24-08-28 05:26:18 grid_pipelin INF   Running on the data_vectors.keys() = dict_keys(['cosmo', 'i_sobol', 'i_example', 'cl', 'i_noise']) 

return_cls = True, return_maps = False: 100%|██████████| 100/100 [01:08<00:00,  1.47it/s]

return_cls = False, return_maps = True
24-08-28 05:27:27     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 


24-08-28 05:27:27     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-28 05:27:27 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-28 05:27:27 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-28 05:27:27 grid_pipelin INF   Including noise_indices = [0] 
24-08-28 05:27:27 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-28 05:27:27 grid_pipelin INF   Interleaving with n_readers = 8 
dict_keys(['cosmo', 'i_sobol', 'i_example', 'kg', 'dg'])
24-08-28 05:27:27 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-28 05:27:27 grid_pipelin INF   Batching into 32 elements locally 
24-08-28 05:27:27 grid_pipelin WAR   Tracing _augmentations 
24-08-28 05:27:27 grid_pipelin INF   Running on the data_vectors.keys() = dict_keys(['cosmo', 'i_sobol', 'i_example', 'kg', 'dg', 

return_cls = False, return_maps = True: 100%|██████████| 100/100 [01:42<00:00,  1.03s/it]


In [24]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
)

data_vec_pix = grid_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = grid_pipe.n_z_metacal + grid_pipe.n_z_maglim
n_z_metacal = grid_pipe.n_z_metacal
n_noise = conf["analysis"]["grid"]["n_noise_per_example"]

grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=128,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

24-08-15 23:51:20     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-15 23:51:20     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-15 23:51:20 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-15 23:51:20 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-15 23:51:20 grid_pipelin INF   Including noise_indices = [0, 1] 
24-08-15 23:51:20 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-15 23:51:20 grid_pipelin INF   Interleaving with n_readers = 1 
24-08-15 23:51:20 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-15 23:51:20 grid_pipelin INF   Batching into 128 elements locally 
24-08-15 23:51:20 grid_pipelin WAR   Tracing _augmentations 
24-08-15 23:51:20 grid_pipe

In [ ]:
%%time
n_batches = 100
for x in tqdm(grid_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [03:35<00:00,  2.14s/it]

In [10]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=False,
)

data_vec_pix = grid_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = grid_pipe.n_z_metacal + grid_pipe.n_z_maglim
n_z_metacal = grid_pipe.n_z_metacal
n_noise = conf["analysis"]["grid"]["n_noise_per_example"]

grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=128,
    n_readers=1,
    n_prefetch=0,
    is_eval=False
)

24-08-16 00:02:00     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:02:01     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:02:01 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-16 00:02:01 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-16 00:02:01 grid_pipelin INF   Including noise_indices = [0, 1] 
24-08-16 00:02:01 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-16 00:02:01 grid_pipelin INF   Interleaving with n_readers = 1 
cosmo
i_sobol
i_example
24-08-16 00:02:01 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-16 00:02:01 grid_pipelin INF   Batching into 128 elements locally 
24-08-16 00:02:01 grid_pipelin WAR   Tracing _augmentations 
24-

In [11]:
%%time
n_batches = 100
for x in tqdm(grid_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [00:58<00:00,  1.70it/s]

CPU times: user 1min 2s, sys: 38.6 s, total: 1min 40s
Wall time: 58.7 s
